In [0]:
# Databricks Notebook source

# MAGIC %md
# MAGIC # 🔧 CONFIGURAÇÃO DO AMBIENTE - EXECUTAR PRIMEIRO!

# COMMAND ----------

# MAGIC %pip install --upgrade --force-reinstall yfinance pandas numpy plotly

# COMMAND ----------

# MAGIC %md
# MAGIC ## 🚀 REINICIAR PARA APLICAR INSTALAÇÕES

# COMMAND ----------

dbutils.library.restartPython()

# COMMAND ----------

# MAGIC %md
# MAGIC ## ✅ VERIFICAR INSTALAÇÃO

# COMMAND ----------

print("🔍 Verificando instalações...")

try:
    import yfinance as yf
    print("✅ yfinance instalado!")
except ImportError as e:
    print(f"❌ yfinance não instalado: {e}")

try:
    import pandas as pd
    print("✅ pandas instalado!")
except ImportError as e:
    print(f"❌ pandas não instalado: {e}")

try:
    import numpy as np
    print("✅ numpy instalado!")
except ImportError as e:
    print(f"❌ numpy não instalado: {e}")

try:
    import plotly.graph_objects as go
    print("✅ plotly instalado!")
except ImportError as e:
    print(f"❌ plotly não instalado: {e}")

# COMMAND ----------

# MAGIC %md
# MAGIC # 📊 DASHBOARD FINANCEIRO

# COMMAND ----------

# IMPORTAR BIBLIOTECAS
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

print("🎉 Todas as bibliotecas prontas!")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. ⚙️ CONFIGURAÇÃO

# COMMAND ----------

# Lista de ações para análise
acoes = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN']

# Período de análise
periodo = "1mo"

print(f"🎯 Analisando {len(acoes)} ações")
print(f"📅 Período: {periodo}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. 📈 COLETAR DADOS

# COMMAND ----------

dados_mercado = {}

for acao in acoes:
    try:
        print(f"⬇️  Baixando {acao}...")
        ticker = yf.Ticker(acao)
        dados = ticker.history(period=periodo, auto_adjust=True)
        
        if not dados.empty:
            dados_mercado[acao] = dados
            print(f"   ✅ {acao}: {len(dados)} registros - U$ {dados['Close'].iloc[-1]:.2f}")
        else:
            print(f"   ❌ {acao}: Sem dados disponíveis")
    except Exception as e:
        print(f"   ❌ {acao}: Erro - {e}")

print(f"\n📊 Coleta concluída: {len(dados_mercado)}/{len(acoes)} ações")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4. 📊 GRÁFICO DE PREÇOS

# COMMAND ----------

if dados_mercado:
    grafico = go.Figure()
    
    for acao, dados in dados_mercado.items():
        grafico.add_trace(go.Scatter(
            x=dados.index, 
            y=dados['Close'], 
            name=acao,
            line=dict(width=2)
        ))
    
    grafico.update_layout(
        title='📈 EVOLUÇÃO DOS PREÇOS',
        xaxis_title='Data',
        yaxis_title='Preço (USD)',
        height=500,
        template='plotly_white'
    )
    
    grafico.show()
else:
    print("❌ Nenhum dado disponível para gerar gráficos")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 5. 📋 RELATÓRIO DE ANÁLISE

# COMMAND ----------

if dados_mercado:
    print("📋 RELATÓRIO DE PERFORMANCE")
    print("=" * 50)
    
    for acao, dados in dados_mercado.items():
        preco_atual = dados['Close'].iloc[-1]
        preco_inicial = dados['Close'].iloc[0]
        variacao = ((preco_atual - preco_inicial) / preco_inicial) * 100
        
        # Emojis para indicar performance
        if variacao > 0:
            emoji = "🟢"
        elif variacao < 0:
            emoji = "🔴"
        else:
            emoji = "⚪"
        
        print(f"{emoji} {acao}:")
        print(f"   💰 Preço Atual: U$ {preco_atual:.2f}")
        print(f"   📈 Variação: {variacao:+.2f}%")
        print(f"   📅 Período: {dados.index[0].strftime('%d/%m/%Y')} a {dados.index[-1].strftime('%d/%m/%Y')}")
        print()
        
    print(f"🎯 Análise concluída para {len(dados_mercado)} ativos")
else:
    print("❌ Nenhum dado disponível para análise")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 6. 💾 SALVAR DADOS (VERSÃO SEGURA)

# COMMAND ----------

# MAGIC %sql
# MAGIC -- Criar schema se não existir (execução segura)
# MAGIC CREATE SCHEMA IF NOT EXISTS workspace_analytics;

# COMMAND ----------

# Salvar dados de forma segura
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np

def coletar_dados_fundamentalistas(acao):
    """Coleta dados fundamentalistas completos de uma ação"""
    try:
        ticker = yf.Ticker(acao)
        info = ticker.info
        
        dados_fund = {
            # Informações Básicas
            'empresa_nome': info.get('longName', acao),
            'setor': info.get('sector', 'N/A'),
            'industria': info.get('industry', 'N/A'),
            'pais': info.get('country', 'N/A'),
            'moeda': info.get('currency', 'N/A'),
            
            # Valuation
            'market_cap': info.get('marketCap'),
            'enterprise_value': info.get('enterpriseValue'),
            'trailing_pe': info.get('trailingPE'),
            'forward_pe': info.get('forwardPE'),
            'peg_ratio': info.get('pegRatio'),
            'price_to_book': info.get('priceToBook'),
            'price_to_sales': info.get('priceToSalesTrailing12Months'),
            'ev_to_ebitda': info.get('enterpriseToEbitda'),
            'ev_to_revenue': info.get('enterpriseToRevenue'),
            
            # Dividendos
            'dividend_yield': info.get('dividendYield', 0) * 100 if info.get('dividendYield') else 0,
            'dividend_rate': info.get('dividendRate'),
            'dividend_annual': info.get('dividendRate'),
            'payout_ratio': info.get('payoutRatio'),
            'dividend_5y_avg': info.get('fiveYearAvgDividendYield', 0) * 100 if info.get('fiveYearAvgDividendYield') else 0,
            'last_dividend_date': info.get('lastDividendDate'),
            
            # Lucratividade
            'trailing_eps': info.get('trailingEps'),
            'forward_eps': info.get('forwardEps'),
            'ebitda': info.get('ebitda'),
            'ebitda_margins': info.get('ebitdaMargins', 0) * 100 if info.get('ebitdaMargins') else 0,
            'gross_margins': info.get('grossMargins', 0) * 100 if info.get('grossMargins') else 0,
            'operating_margins': info.get('operatingMargins', 0) * 100 if info.get('operatingMargins') else 0,
            'profit_margins': info.get('profitMargins', 0) * 100 if info.get('profitMargins') else 0,
            'return_on_assets': info.get('returnOnAssets', 0) * 100 if info.get('returnOnAssets') else 0,
            'return_on_equity': info.get('returnOnEquity', 0) * 100 if info.get('returnOnEquity') else 0,
            
            # Crescimento
            'revenue_growth': info.get('revenueGrowth', 0) * 100 if info.get('revenueGrowth') else 0,
            'earnings_growth': info.get('earningsGrowth', 0) * 100 if info.get('earningsGrowth') else 0,
            'earnings_quarterly_growth': info.get('earningsQuarterlyGrowth', 0) * 100 if info.get('earningsQuarterlyGrowth') else 0,
            
            # Balanço Patrimonial
            'total_assets': info.get('totalAssets'),
            'total_cash': info.get('totalCash'),
            'total_debt': info.get('totalDebt'),
            'debt_to_equity': info.get('debtToEquity', 0) * 100 if info.get('debtToEquity') else 0,
            'current_ratio': info.get('currentRatio'),
            'quick_ratio': info.get('quickRatio'),
            
            # Analistas
            'target_mean_price': info.get('targetMeanPrice'),
            'target_high_price': info.get('targetHighPrice'),
            'target_low_price': info.get('targetLowPrice'),
            'recommendation_mean': info.get('recommendationMean'),
            'number_of_analyst_opinions': info.get('numberOfAnalystOpinions'),
            'recommendation_key': info.get('recommendationKey', 'N/A'),
            
            # Dados do Negócio
            'volume_24h': info.get('volume24Hr'),
            'average_volume': info.get('averageVolume'),
            'average_volume_10days': info.get('averageVolume10days'),
            'fifty_two_week_high': info.get('fiftyTwoWeekHigh'),
            'fifty_two_week_low': info.get('fiftyTwoWeekLow'),
            'fifty_day_average': info.get('fiftyDayAverage'),
            'two_hundred_day_average': info.get('twoHundredDayAverage'),
            
            # Data de Coleta
            'data_coleta_fundamentalista': datetime.now()
        }
        
        # Calcular scores fundamentalistas
        dados_fund['score_valuation'] = calcular_score_valuation(dados_fund)
        dados_fund['score_lucratividade'] = calcular_score_lucratividade(dados_fund)
        dados_fund['score_crescimento'] = calcular_score_crescimento(dados_fund)
        dados_fund['score_dividendos'] = calcular_score_dividendos(dados_fund)
        dados_fund['score_fundamentalista_total'] = (
            dados_fund['score_valuation'] + 
            dados_fund['score_lucratividade'] + 
            dados_fund['score_crescimento'] + 
            dados_fund['score_dividendos']
        )
        
        return dados_fund
        
    except Exception as e:
        print(f"❌ Erro ao coletar dados fundamentalistas para {acao}: {e}")
        return None

def calcular_score_valuation(dados):
    """Calcula score de valuation (0-10)"""
    score = 0
    try:
        if dados.get('trailing_pe') and dados['trailing_pe'] < 15: score += 3
        elif dados.get('trailing_pe') and dados['trailing_pe'] < 25: score += 2
        elif dados.get('trailing_pe') and dados['trailing_pe'] < 35: score += 1
        
        if dados.get('price_to_book') and dados['price_to_book'] < 1.5: score += 2
        elif dados.get('price_to_book') and dados['price_to_book'] < 3: score += 1
        
        if dados.get('ev_to_ebitda') and dados['ev_to_ebitda'] < 10: score += 2
        elif dados.get('ev_to_ebitda') and dados['ev_to_ebitda'] < 15: score += 1
        
        if dados.get('peg_ratio') and dados['peg_ratio'] < 1: score += 3
        elif dados.get('peg_ratio') and dados['peg_ratio'] < 1.5: score += 2
        elif dados.get('peg_ratio') and dados['peg_ratio'] < 2: score += 1
        
    except Exception as e:
        print(f"⚠️ Erro no cálculo de valuation: {e}")
    
    return min(score, 10)

def calcular_score_lucratividade(dados):
    """Calcula score de lucratividade (0-10)"""
    score = 0
    try:
        if dados.get('return_on_equity') and dados['return_on_equity'] > 20: score += 3
        elif dados.get('return_on_equity') and dados['return_on_equity'] > 15: score += 2
        elif dados.get('return_on_equity') and dados['return_on_equity'] > 10: score += 1
        
        if dados.get('profit_margins') and dados['profit_margins'] > 20: score += 3
        elif dados.get('profit_margins') and dados['profit_margins'] > 15: score += 2
        elif dados.get('profit_margins') and dados['profit_margins'] > 10: score += 1
        
        if dados.get('operating_margins') and dados['operating_margins'] > 25: score += 2
        elif dados.get('operating_margins') and dados['operating_margins'] > 15: score += 1
        
        if dados.get('ebitda_margins') and dados['ebitda_margins'] > 30: score += 2
        elif dados.get('ebitda_margins') and dados['ebitda_margins'] > 20: score += 1
        
    except Exception as e:
        print(f"⚠️ Erro no cálculo de lucratividade: {e}")
    
    return min(score, 10)

def calcular_score_crescimento(dados):
    """Calcula score de crescimento (0-10)"""
    score = 0
    try:
        if dados.get('revenue_growth') and dados['revenue_growth'] > 20: score += 3
        elif dados.get('revenue_growth') and dados['revenue_growth'] > 10: score += 2
        elif dados.get('revenue_growth') and dados['revenue_growth'] > 5: score += 1
        
        if dados.get('earnings_growth') and dados['earnings_growth'] > 25: score += 3
        elif dados.get('earnings_growth') and dados['earnings_growth'] > 15: score += 2
        elif dados.get('earnings_growth') and dados['earnings_growth'] > 5: score += 1
        
        if dados.get('earnings_quarterly_growth') and dados['earnings_quarterly_growth'] > 20: score += 2
        elif dados.get('earnings_quarterly_growth') and dados['earnings_quarterly_growth'] > 10: score += 1
        
    except Exception as e:
        print(f"⚠️ Erro no cálculo de crescimento: {e}")
    
    return min(score, 10)

def calcular_score_dividendos(dados):
    """Calcula score de dividendos (0-10)"""
    score = 0
    try:
        if dados.get('dividend_yield') and dados['dividend_yield'] > 5: score += 3
        elif dados.get('dividend_yield') and dados['dividend_yield'] > 3: score += 2
        elif dados.get('dividend_yield') and dados['dividend_yield'] > 1: score += 1
        
        if dados.get('payout_ratio') and dados['payout_ratio'] < 0.5: score += 2
        elif dados.get('payout_ratio') and dados['payout_ratio'] < 0.7: score += 1
        
        if dados.get('dividend_5y_avg') and dados['dividend_5y_avg'] > 3: score += 2
        elif dados.get('dividend_5y_avg') and dados['dividend_5y_avg'] > 1: score += 1
        
    except Exception as e:
        print(f"⚠️ Erro no cálculo de dividendos: {e}")
    
    return min(score, 10)

# EXECUÇÃO PRINCIPAL
print("🚀 INICIANDO COLETA DE DADOS FUNDAMENTALISTAS...")

# Lista de ações do seu CSV
acoes = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN']

dados_fundamentalistas = []

for acao in acoes:
    print(f"📊 Coletando dados fundamentalistas para {acao}...")
    
    dados_fund = coletar_dados_fundamentalistas(acao)
    if dados_fund:
        dados_fund['symbol'] = acao
        dados_fundamentalistas.append(dados_fund)
        print(f"✅ Dados coletados para {acao}")
    else:
        print(f"❌ Falha na coleta para {acao}")

# Criar DataFrame com dados fundamentalistas
if dados_fundamentalistas:
    df_fundamentalista = pd.DataFrame(dados_fundamentalistas)
    
    # Salvar no Databricks
    spark_df = spark.createDataFrame(df_fundamentalista)
    
    # Configurar ambiente
    spark.sql("USE CATALOG workspace_analise")
    spark.sql("USE analytics")
    
    # Salvar tabela
    spark_df.write.mode("overwrite").saveAsTable("dashboard_fundamentalista")
    
    print("✅ DADOS FUNDAMENTALISTAS SALVOS COM SUCESSO!")
    
    # Mostrar dashboard
    print("\n🎯 DASHBOARD FUNDAMENTALISTA:")
    display(spark.sql("""
        SELECT 
            symbol,
            empresa_nome,
            setor,
            ROUND(market_cap/1000000000, 2) as market_cap_bilhoes,
            ROUND(trailing_pe, 1) as pe_ratio,
            ROUND(dividend_yield, 2) as dividend_yield,
            ROUND(return_on_equity, 1) as roe,
            ROUND(profit_margins, 1) as margem_liquida,
            score_fundamentalista_total as score_total,
            recommendation_key as recomendacao
        FROM dashboard_fundamentalista 
        ORDER BY score_fundamentalista_total DESC
    """))
    
else:
    print("❌ Nenhum dado fundamentalista foi coletado")
# COMMAND ----------

# MAGIC %md
# MAGIC # 🎉 DASHBOARD CONCLUÍDO!

# COMMAND ----------

print("""
✅ DASHBOARD EXECUTADO COM SUCESSO!

📊 Funcionalidades implementadas:
- ✅ Coleta de dados do mercado
- ✅ Gráficos interativos
- ✅ Análise de performance  
- ✅ Armazenamento em banco de dados

🚀 Próximos passos:
- Adicionar mais indicadores técnicos
- Criar alertas automáticos
- Análise preditiva
""")